In [18]:
!pip install python-binance

import warnings
warnings.filterwarnings('ignore')





Defaulting to user installation because normal site-packages is not writeable


In [32]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import os
from binance.client import Client

apiKey = "urN4NPZbcuV3mSsmFFD7pJxpsJxxZWmNkjO3EAMyMRuHpKwnJltGXbIXfZLzokyg"
apiSecret_Key = "Pu6nipFs2rEglu4t38SmPlNn7g8jI9nrXybTIxDrJhydkyRIzAdPuyREj0DMirOo"

client = Client(apiKey, apiSecret_Key, tld='com')

# Definir los parámetros de la consulta
symbol = 'BTCUSDT'  # Par de trading (puedes cambiarlo según tus necesidades)
interval = Client.KLINE_INTERVAL_1HOUR  # Intervalo de tiempo de la candela (1 hora en este caso)
limit = 100 # Número máximo de candelas a obtener

# Obtener los datos de kline
klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)

# Imprimir los datos obtenidos
print(klines)

[[1708012800000, '51948.24000000', '52380.00000000', '51742.00000000', '52265.32000000', '3408.06139000', 1708016399999, '177573766.99237950', 92715, '1799.01723000', '93755399.82759470', '0'], [1708016400000, '52265.32000000', '52317.30000000', '51850.50000000', '51875.59000000', '2173.95966000', 1708019999999, '113340263.09503000', 72235, '983.47187000', '51282737.15626200', '0'], [1708020000000, '51875.60000000', '52060.00000000', '51831.34000000', '51977.99000000', '1611.13065000', 1708023599999, '83709835.14423080', 62480, '859.00669000', '44630231.08288830', '0'], [1708023600000, '51977.98000000', '52056.00000000', '51725.00000000', '52002.01000000', '2065.22624000', 1708027199999, '107184143.81505720', 59991, '999.91648000', '51902649.41038070', '0'], [1708027200000, '52002.01000000', '52011.10000000', '51569.81000000', '51686.00000000', '2961.36308000', 1708030799999, '153356081.41541370', 70654, '1369.61914000', '70926306.37806590', '0'], [1708030800000, '51686.00000000', '518

In [33]:
# Convertir la lista de listas en un DataFrame
df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
# Seleccionar solo las columnas deseadas
df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# Convertir los valores a numéricos
df = df.apply(pd.to_numeric)

# Convertir los timestamps a formato de fecha y hora
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Convertir los valores a numéricos
df = df.apply(pd.to_numeric, errors='ignore')

# Mostrar el DataFrame
print(df)

              timestamp      open      high       low     close      volume
0   1708012800000000000  51948.24  52380.00  51742.00  52265.32  3408.06139
1   1708016400000000000  52265.32  52317.30  51850.50  51875.59  2173.95966
2   1708020000000000000  51875.60  52060.00  51831.34  51977.99  1611.13065
3   1708023600000000000  51977.98  52056.00  51725.00  52002.01  2065.22624
4   1708027200000000000  52002.01  52011.10  51569.81  51686.00  2961.36308
..                  ...       ...       ...       ...       ...         ...
95  1708354800000000000  52100.00  52346.05  52028.93  52127.99  1349.87979
96  1708358400000000000  52127.99  52230.00  51677.00  51775.76  2178.19956
97  1708362000000000000  51775.76  52210.00  51688.69  52151.73  1971.50334
98  1708365600000000000  52151.72  52200.00  51913.07  51926.61  1131.63654
99  1708369200000000000  51926.62  51933.79  51781.10  51819.75   648.12575

[100 rows x 6 columns]


In [34]:
#Separamos los datos entre X e y
X = df[['timestamp', 'open', 'high', 'low', 'volume']]
y = df['close']

#Separamos los datos entre entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [35]:
#Entrenando el modelo
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [36]:
#Recuperamos la intersección
print(regressor.intercept_)

77432.27025770876


In [37]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
timestamp,-4.425050e-14
open,-5.871175e-01
high,6.515029e-01
low,9.004703e-01
volume,2.259482e-02


In [38]:
y_pred = regressor.predict(X_test)
y_pred

array([51895.66119386, 52397.24093392, 52020.68521546, 51660.80895292,
       51789.00602528, 52233.5233857 , 51937.73406043, 51924.72423699,
       51697.26269776, 52232.50440309, 51863.88580056, 52345.88823205,
       52236.05119722, 52112.1232098 , 51907.51742515, 51938.73312862,
       51987.39995735, 52147.9740082 , 51907.89578823, 51943.93393593])

In [39]:
#Convertimos en df la salida
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
26,51967.33,51895.661194
86,52387.53,52397.240934
2,51977.99,52020.685215
55,51642.64,51660.808953
75,51758.12,51789.006025
93,52150.17,52233.523386
16,51829.30,51937.734060
73,51837.94,51924.724237
54,51710.51,51697.262698
95,52127.99,52232.504403


In [40]:
error = np.sqrt(mean_squared_error(y_test, y_pred))
print("Error porcentual : %f" % (error*100))

Error porcentual : 6336.346995


In [41]:
r2_score(y_test, y_pred)

0.88641983020633

In [42]:
df.head()

,Actual,Predicted
26,51967.33,51895.661194
86,52387.53,52397.240934
2,51977.99,52020.685215
55,51642.64,51660.808953
75,51758.12,51789.006025


In [43]:
close_prediction = pd.DataFrame(np.array([[1708304400000000000,51613.38,51895.23,51526.66,2449]]),columns=['timestamp', 'open', 'high', 'low', 'volume'])
close_prediction

,timestamp,open,high,low,volume
0,1.708304e+18,51613.38,51895.23,51526.66,2449.0


In [44]:
regressor.predict(close_prediction)

array([51799.28239651])